<a href="https://colab.research.google.com/github/isabellasims/Google-colabs/blob/master/geopandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
!pip install geopandas
import geopandas as gpd #rename as gpd
import math

In [3]:
import folium
from folium import Choropleth, Circle, Marker
from folium.plugins import HeatMap, MarkerCluster

Several arguments customize the appearance of the map:

location sets the initial center of the map. We use the latitude (42.32° N) and longitude (-71.0589° E) of the city of Boston.
tiles changes the styling of the map; in this case, we choose the OpenStreetMap style. If you're curious, you can find the other options listed here.
zoom_start sets the initial level of zoom of the map, where higher values zoom in closer to the map.
Take the time now to explore by zooming in and out, or by dragging the map in different directions.

In [ ]:
# Create a map

# location = lat long dc
# tiles changes the styling of the map
m_1 = folium.Map(location=[38.9072,-77.0369], tiles='openstreetmap', zoom_start=10)


# Display the map
m_1

# https://www.kaggle.com/alexisbcook/interactive-maps

In [49]:
# Load the data
# data from http://crimemap.dc.gov/Download.aspx
crimes = pd.read_csv("/content/SearchResults.txt", encoding='latin-1')

# Drop rows with missing locations
#crimes.dropna(subset=['Lat', 'Long', 'DISTRICT'], inplace=True)

# Focus on major crimes in 2018
crimes = crimes[crimes.OFFENSE.isin([
    'ROBBERY','ASSAULT W/DANGEROUS WEAPON', 'HOMICIDE','SEX ABUSE'])]
#crimes = crimes[crimes.REPORT_DATE>=2018]
# address = crimes.BLOCK[]
# address.head()

# Print the first five rows of the table
crimes.head()

,REPORT_DAT,OFFENSE,METHOD,BLOCK,DISTRICT,WARD,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,XBLOCK,YBLOCK,START_DATE
3,6/30/2020 1:03:01 AM,ROBBERY,GUN,2922 - 2999 BLOCK OF 2ND STREET SE,7.0,8,Cluster 39,010400 2,399591.0,130876.0,6/29/2020 11:17:32 PM
4,6/30/2020 1:10:06 AM,ROBBERY,OTHERS,200 - 289 BLOCK OF TAYLOR STREET NE,4.0,5,Cluster 19,009501 3,399680.0,141427.0,6/29/2020 11:47:39 PM
9,6/30/2020 3:13:25 AM,ASSAULT W/DANGEROUS WEAPON,GUN,4100 - 4199 BLOCK OF BENNING ROAD NE,6.0,7,Cluster 32,009603 1,404896.0,136155.0,6/30/2020 1:39:14 AM
10,7/1/2020 1:05:07 AM,ROBBERY,OTHERS,3500 - 3899 BLOCK OF JAY STREET NE,6.0,7,Cluster 30,009602 2,404579.0,137347.0,6/30/2020 2:00:15 AM
15,6/30/2020 7:35:56 AM,ROBBERY,GUN,3500 - 3703 BLOCK OF WHEELER ROAD SE,7.0,8,Cluster 39,009804 1,400557.0,130171.0,6/30/2020 5:18:24 AM


In [27]:
homicides = crimes[crimes.OFFENSE == 'HOMICIDE']
homicides

,REPORT_DAT,OFFENSE,METHOD,BLOCK,DISTRICT,WARD,NEIGHBORHOOD_CLUSTER,BLOCK_GROUP,XBLOCK,YBLOCK,START_DATE


In [ ]:
# Create a map
m_2 = folium.Map(location=[38.9072,-77.0369], tiles='cartodbpositron', zoom_start=13)

# Add points to the map
for idx, row in crimes.iterrows():
    Marker([row['XBLOCK'], row['YBLOCK']]).add_to(m_2)

# Display the map
m_2

In [45]:
from geopy.geocoders import Nominatim
geolocator = Nominatim(timeout=10, user_agent = "dlab.berkeley.edu-workshop")

location = geolocator.geocode('Ward 8 Washington DC')
location

Location(DC, Dương Bá Trạc, Phường 2, Quận 8, Thành phố Hồ Chí Minh, Long Điền, 710400, Việt Nam, (10.7456759, 106.6892684, 0.0))